# Retrieving HG38 epigenomic files
The following notebook shows how the epigenomic files metadata are retrieved.

In [1]:
from glob import glob
import pandas as pd
import compress_json
from encodeproject import biosamples, accessions, biosample, download_urls

We specify that we are only interested in the [GRCh38](https://www.ncbi.nlm.nih.gov/assembly/GCF_000001405.26/) assembly, are currently in status [released](https://www.encodeproject.org/help/getting-started/status-terms/#FileStatuses), have replication type [isogenic](https://www.encodeproject.org/data-standards/terms/) (there is a biological replication) and the file format is [bigWig](https://genome.ucsc.edu/goldenPath/help/bigWig.html#:~:text=The%20bigWig%20format%20is%20useful,in%20an%20indexed%20binary%20format.&text=Wiggle%20data%20must%20be%20continuous%20and%20consist%20of%20equally%20sized%20elements.).

In [2]:
parameters = dict(
    assembly="GRCh38",
    replication_type="isogenic",
    file_format="bigWig",
    status="released"
)

We will append all the dataset while we obtain them to the following list.

In [3]:
all_datasets = []

### Retrieving CHIP-seq

In [4]:
samples = biosamples(
    accessions=accessions(compress_json.load("hg38_encode_queries/chipseq.json")),
    min_biological_replicates=2,
    output_type="fold change over control",
    **parameters
)
all_datasets.append(samples)
samples

,organism,target,term_id,cell_line,institute_name,title,accession,status,assay_title,assay_term_name,...,output_type,read_length,read_length_units,run_type,schema_version,date_created,pipeline_award_rfas,biological_replicates,technical_replicates,url
0,human,H3K4me2,EFO:0001203,MCF-7,Broad Institute,"Bradley Bernstein, Broad",ENCFF299JCQ,released,Histone ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:16:20.480527+00:00]",[ENCODE3],"[1, 2]","[1_1, 1_2, 2_1, 2_2, 2_3]",https://encode-public.s3.amazonaws.com/2016/11...
1,human,SIX5,EFO:0003042,H1,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF665USC,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-10-14, 00:44:09.736803+00:00]",[ENCODE3],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2016/12...
2,human,HCFC1,EFO:0002067,K562,Stanford University,"Michael Snyder, Stanford",ENCFF339TEC,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-10-14, 00:44:01.750808+00:00]",[ENCODE3],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2016/12...
3,human,MYC,EFO:0002067,K562,Stanford University,"Michael Snyder, Stanford",ENCFF677COF,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:44:11.329673+00:00]",[ENCODE3],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2016/12...
4,human,ZNF274,EFO:0002067,K562,University of Southern California,"Peggy Farnham, USC",ENCFF767MPH,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:39:01.456813+00:00]",[ENCODE3],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2017/02...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767,human,CTCF,EFO:0002067,K562,Stanford University,"Michael Snyder, Stanford",ENCFF050CCI,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:39:02.683242+00:00]",[ENCODE3],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2016/11...
1768,human,CTCF,EFO:0002067,K562,Stanford University,"Michael Snyder, Stanford",ENCFF682MFJ,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-09-25, 20:54:07.885028+00:00]",[ENCODE4],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2020/09...
1769,human,EP300,EFO:0001187,HepG2,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF760WAX,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 04:39:17.087390+00:00]",[ENCODE4],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2020/08...
1770,human,POLR2A,EFO:0002784,GM12878,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF600UCV,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:32:02.326569+00:00]",[ENCODE3],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2017/01...


In [5]:
samples.groupby(["organism", "target", "cell_line"]).agg({'pipeline_award_rfas': lambda x: [
    i
    for e in x
    for i in e
]}).pipeline_award_rfas.apply(lambda x: x==["ENCODE3"]).sum()

617

In [6]:
617 / 1772

0.3481941309255079

### Retrieving DNASE-seq

In [7]:
samples = biosamples(
    accessions=accessions(compress_json.load("hg38_encode_queries/dnaseseq.json")),
    organism=None,
    **parameters
)
samples["organism"] = "human"

all_datasets.append(samples)
samples

,organism,target,term_id,cell_line,institute_name,title,accession,status,assay_title,assay_term_name,...,output_type,read_length,read_length_units,run_type,schema_version,date_created,pipeline_award_rfas,biological_replicates,technical_replicates,url
0,human,Unknown,EFO:0001187,HepG2,University of Washington,"John Stamatoyannopoulos, UW",ENCFF113VII,released,DNase-seq,DNase-seq,...,read-depth normalized signal,NaN,None,None,25,"[2020-09-24, 17:13:26.629062+00:00]",[ENCODE4],[2],[2_1],https://encode-public.s3.amazonaws.com/2020/09...
1,human,Unknown,EFO:0001187,HepG2,University of Washington,"John Stamatoyannopoulos, UW",ENCFF577SOF,released,DNase-seq,DNase-seq,...,read-depth normalized signal,NaN,None,None,25,"[2020-09-23, 05:09:31.096867+00:00]",[ENCODE3],[1],[1_1],https://encode-public.s3.amazonaws.com/2017/08...
2,human,Unknown,EFO:0001187,HepG2,University of Washington,"John Stamatoyannopoulos, UW",ENCFF842XRQ,released,DNase-seq,DNase-seq,...,read-depth normalized signal,NaN,None,None,25,"[2020-09-23, 05:09:31.096867+00:00]",[ENCODE3],[2],[2_1],https://encode-public.s3.amazonaws.com/2017/08...
3,human,Unknown,EFO:0001187,HepG2,University of Washington,"John Stamatoyannopoulos, UW",ENCFF546MZK,released,DNase-seq,DNase-seq,...,read-depth normalized signal,NaN,None,None,25,"[2020-09-24, 17:13:26.629062+00:00]",[ENCODE4],[1],[1_1],https://encode-public.s3.amazonaws.com/2020/09...
4,human,Unknown,EFO:0002067,K562,University of Washington,"John Stamatoyannopoulos, UW",ENCFF413AHU,released,DNase-seq,DNase-seq,...,read-depth normalized signal,NaN,None,None,25,"[2020-11-09, 19:41:15.294787+00:00]",[ENCODE3],[1],"[1_1, 1_2]",https://encode-public.s3.amazonaws.com/2018/05...
5,human,Unknown,EFO:0002067,K562,University of Washington,"John Stamatoyannopoulos, UW",ENCFF936BDN,released,DNase-seq,DNase-seq,...,read-depth normalized signal,NaN,None,None,25,"[2020-11-09, 19:41:15.294787+00:00]",[ENCODE3],[2],[2_1],https://encode-public.s3.amazonaws.com/2018/05...
6,human,Unknown,EFO:0001203,MCF-7,University of Washington,"John Stamatoyannopoulos, UW",ENCFF924FJR,released,DNase-seq,DNase-seq,...,read-depth normalized signal,NaN,None,None,25,"[2020-09-23, 05:09:26.911315+00:00]",[ENCODE3],[1],[1_1],https://encode-public.s3.amazonaws.com/2017/08...
7,human,Unknown,EFO:0001203,MCF-7,University of Washington,"John Stamatoyannopoulos, UW",ENCFF949ANK,released,DNase-seq,DNase-seq,...,read-depth normalized signal,NaN,None,None,25,"[2020-09-23, 05:09:26.911315+00:00]",[ENCODE3],[2],[2_1],https://encode-public.s3.amazonaws.com/2017/08...
8,human,Unknown,EFO:0001203,MCF-7,University of Washington,"John Stamatoyannopoulos, UW",ENCFF631QXL,released,DNase-seq,DNase-seq,...,read-depth normalized signal,NaN,None,None,25,"[2020-09-24, 17:12:01.866061+00:00]",[ENCODE4],[1],[1_1],https://encode-public.s3.amazonaws.com/2020/09...
9,human,Unknown,EFO:0001203,MCF-7,University of Washington,"John Stamatoyannopoulos, UW",ENCFF134COA,released,DNase-seq,DNase-seq,...,read-depth normalized signal,NaN,None,None,25,"[2020-09-24, 17:12:01.866061+00:00]",[ENCODE4],[2],[2_1],https://encode-public.s3.amazonaws.com/2020/09...


### Retrieving WGBS

In [8]:
samples = biosamples(
    accessions=accessions(compress_json.load("hg38_encode_queries/wgbs.json")),
    organism=None,
    **parameters
)
samples["organism"] = "human"
all_datasets.append(samples)
samples

,organism,target,term_id,cell_line,institute_name,title,accession,status,assay_title,assay_term_name,...,output_type,read_length,read_length_units,run_type,schema_version,date_created,pipeline_award_rfas,biological_replicates,technical_replicates,url
0,human,Unknown,EFO:0001187,HepG2,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF400QTE,released,WGBS,whole-genome shotgun bisulfite sequencing,...,signal,NaN,None,None,25,None,None,[1],[1_1],https://encode-public.s3.amazonaws.com/2017/11...
1,human,Unknown,EFO:0001187,HepG2,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF583VWF,released,WGBS,whole-genome shotgun bisulfite sequencing,...,signal,NaN,None,None,25,None,None,[2],[2_1],https://encode-public.s3.amazonaws.com/2017/11...
2,human,Unknown,EFO:0001086,A549,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF527HXB,released,WGBS,whole-genome shotgun bisulfite sequencing,...,signal,NaN,None,None,25,None,None,[1],[1_1],https://encode-public.s3.amazonaws.com/2017/12...
3,human,Unknown,EFO:0001086,A549,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF401QUB,released,WGBS,whole-genome shotgun bisulfite sequencing,...,signal,NaN,None,None,25,None,None,[2],[2_1],https://encode-public.s3.amazonaws.com/2017/12...
4,human,Unknown,EFO:0002784,GM12878,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF796NFQ,released,WGBS,whole-genome shotgun bisulfite sequencing,...,signal,NaN,None,None,25,None,None,[1],[1_1],https://encode-public.s3.amazonaws.com/2016/03...
5,human,Unknown,EFO:0002784,GM12878,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF812CHG,released,WGBS,whole-genome shotgun bisulfite sequencing,...,signal,NaN,None,None,25,None,None,[2],[2_1],https://encode-public.s3.amazonaws.com/2016/03...
6,human,Unknown,EFO:0001187,HepG2,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF073DUG,released,WGBS,whole-genome shotgun bisulfite sequencing,...,signal,NaN,None,None,25,None,None,[1],[1_1],https://encode-public.s3.amazonaws.com/2016/03...
7,human,Unknown,EFO:0001187,HepG2,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF716NKX,released,WGBS,whole-genome shotgun bisulfite sequencing,...,signal,NaN,None,None,25,None,None,[2],[2_1],https://encode-public.s3.amazonaws.com/2016/03...
8,human,Unknown,EFO:0002067,K562,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF872YSC,released,WGBS,whole-genome shotgun bisulfite sequencing,...,signal,NaN,None,None,25,None,None,[1],[1_1],https://encode-public.s3.amazonaws.com/2016/03...
9,human,Unknown,EFO:0002067,K562,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF669KCI,released,WGBS,whole-genome shotgun bisulfite sequencing,...,signal,NaN,None,None,25,None,None,[2],[2_1],https://encode-public.s3.amazonaws.com/2016/03...


### Retrieving ATAC

In [9]:
samples = biosamples(
    accessions=accessions(compress_json.load("hg38_encode_queries/atacseq.json")),
    organism=None,
    min_biological_replicates=2,
    output_type="fold change over control",
    **parameters
)
samples["organism"] = "human"

all_datasets.append(samples)
samples

,organism,target,term_id,cell_line,institute_name,title,accession,status,assay_title,assay_term_name,...,output_type,read_length,read_length_units,run_type,schema_version,date_created,pipeline_award_rfas,biological_replicates,technical_replicates,url
0,human,Unknown,EFO:0002784,GM12878,Stanford University,"Michael Snyder, Stanford",ENCFF091DNE,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 04:40:16.463642+00:00]",[ENCODE4],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2020/07...
1,human,Unknown,EFO:0001203,MCF-7,Stanford University,"Michael Snyder, Stanford",ENCFF927YYP,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 04:42:20.326375+00:00]",[ENCODE4],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2020/07...
2,human,Unknown,EFO:0002067,K562,Stanford University,"Michael Snyder, Stanford",ENCFF831OKQ,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 04:46:49.876893+00:00]",[ENCODE4],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2020/07...
3,human,Unknown,EFO:0001187,HepG2,Stanford University,"Michael Snyder, Stanford",ENCFF872HBM,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:04:46.772633+00:00]",[ENCODE4],"[1, 2, 3]","[1_1, 2_1, 3_1]",https://encode-public.s3.amazonaws.com/2020/07...
4,human,Unknown,EFO:0002784,GM12878,Stanford University,"Michael Snyder, Stanford",ENCFF948GBJ,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 04:45:28.380581+00:00]",[ENCODE4],"[1, 2, 3]","[1_1, 2_1, 3_1]",https://encode-public.s3.amazonaws.com/2020/07...
5,human,Unknown,EFO:0002067,K562,Stanford University,"Michael Snyder, Stanford",ENCFF450XLU,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 04:37:41.983348+00:00]",[ENCODE4],"[1, 2, 3]","[1_1, 2_1, 3_1]",https://encode-public.s3.amazonaws.com/2020/07...
6,human,Unknown,EFO:0001086,A549,Stanford University,"Michael Snyder, Stanford",ENCFF225UNA,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:01:59.268914+00:00]",[ENCODE4],"[1, 2, 3]","[1_1, 2_1, 3_1]",https://encode-public.s3.amazonaws.com/2020/07...


## Merging all the datasets

In [10]:
pd.concat(all_datasets)

,organism,target,term_id,cell_line,institute_name,title,accession,status,assay_title,assay_term_name,...,output_type,read_length,read_length_units,run_type,schema_version,date_created,pipeline_award_rfas,biological_replicates,technical_replicates,url
0,human,H3K4me2,EFO:0001203,MCF-7,Broad Institute,"Bradley Bernstein, Broad",ENCFF299JCQ,released,Histone ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:16:20.480527+00:00]",[ENCODE3],"[1, 2]","[1_1, 1_2, 2_1, 2_2, 2_3]",https://encode-public.s3.amazonaws.com/2016/11...
1,human,SIX5,EFO:0003042,H1,HudsonAlpha Institute for Biotechnology,"Richard Myers, HAIB",ENCFF665USC,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-10-14, 00:44:09.736803+00:00]",[ENCODE3],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2016/12...
2,human,HCFC1,EFO:0002067,K562,Stanford University,"Michael Snyder, Stanford",ENCFF339TEC,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-10-14, 00:44:01.750808+00:00]",[ENCODE3],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2016/12...
3,human,MYC,EFO:0002067,K562,Stanford University,"Michael Snyder, Stanford",ENCFF677COF,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:44:11.329673+00:00]",[ENCODE3],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2016/12...
4,human,ZNF274,EFO:0002067,K562,University of Southern California,"Peggy Farnham, USC",ENCFF767MPH,released,TF ChIP-seq,ChIP-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:39:01.456813+00:00]",[ENCODE3],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2017/02...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,human,Unknown,EFO:0002067,K562,Stanford University,"Michael Snyder, Stanford",ENCFF831OKQ,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 04:46:49.876893+00:00]",[ENCODE4],"[1, 2]","[1_1, 2_1]",https://encode-public.s3.amazonaws.com/2020/07...
3,human,Unknown,EFO:0001187,HepG2,Stanford University,"Michael Snyder, Stanford",ENCFF872HBM,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 05:04:46.772633+00:00]",[ENCODE4],"[1, 2, 3]","[1_1, 2_1, 3_1]",https://encode-public.s3.amazonaws.com/2020/07...
4,human,Unknown,EFO:0002784,GM12878,Stanford University,"Michael Snyder, Stanford",ENCFF948GBJ,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 04:45:28.380581+00:00]",[ENCODE4],"[1, 2, 3]","[1_1, 2_1, 3_1]",https://encode-public.s3.amazonaws.com/2020/07...
5,human,Unknown,EFO:0002067,K562,Stanford University,"Michael Snyder, Stanford",ENCFF450XLU,released,ATAC-seq,ATAC-seq,...,fold change over control,NaN,None,None,25,"[2020-09-05, 04:37:41.983348+00:00]",[ENCODE4],"[1, 2, 3]","[1_1, 2_1, 3_1]",https://encode-public.s3.amazonaws.com/2020/07...
